In [51]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [52]:
def extract_2d_array(tensor_str):
    
    tensor_str = tensor_str.replace('tensor(','').replace('grad_fn=<TanhBackward0>)', '').replace(' ', '').replace('\n', '')
    tensor_str = tensor_str[:len(tensor_str)-1]
    numbers = eval(tensor_str)
    
    array = np.array(numbers).reshape((-1, 768))  
    # print(array)
    
    return array

def is_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False


In [53]:
# df = pd.read_csv("./dataset.csv")
# df = df[:80000]
# df.to_csv("dataset_80000.csv", index=False)

In [54]:
df = pd.read_csv("./dataset_80000.csv")

df = df[df['ratings'].apply(is_float) & df['no_of_ratings'].apply(is_float)]

print(df['ratings'].unique())
print(df['no_of_ratings'].unique())
df['ratings'] = df['ratings'].astype('float64')
df['no_of_ratings'] = df['no_of_ratings'].astype('int')

grouped = df.groupby(by=['sub_category'])
ratings_mean = grouped['ratings'].mean()
print(ratings_mean)
number_mean = grouped['no_of_ratings'].mean()
print(number_mean)

['4.3' '4.5' '4.4' '3.6' '4.1' '4.2' '3.9' '3.7' '3.8' '4.0' '3.5' '5.0'
 '4.7' '3.3' '4.8' '3.1' '3.0' '4.6' '2.9' '2.8' '3.4' '3.2' '1.3' '2.5'
 '2.2' '4.9' '2.6' '2.7' '1.0' '1.5' '2.4' '2.0' '1.8' '2.3' '1.4' '2.1'
 '1.2' '1.9' '1.6' '1.7' '1.1']
['37' '396' '985' '94' '38' '26' '254' '205' '428' '477' '196' '295' '172'
 '872' '752' '76' '30' '73' '150' '206' '740' '374' '114' '996' '83' '75'
 '822' '377' '96' '63' '703' '93' '22' '29' '557' '27' '874' '224' '882'
 '281' '519' '723' '87' '273' '55' '215' '467' '182' '15' '218' '12' '109'
 '213' '69' '788' '394' '436' '19' '1' '4' '175' '25' '180' '188' '35'
 '323' '184' '2' '34' '495' '3' '492' '223' '77' '155' '336' '241' '126'
 '862' '412' '322' '104' '342' '134' '132' '297' '413' '478' '131' '640'
 '570' '685' '401' '5' '546' '216' '208' '260' '18' '10' '981' '49' '385'
 '11' '60' '398' '632' '473' '775' '165' '423' '623' '431' '421' '14'
 '681' '17' '41' '71' '515' '577' '633' '287' '256' '851' '177' '265'
 '516' '709' '404' '3

In [55]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_no = grouped['no_of_ratings'].quantile(0.25)
# Print the 1/4th quartile values
print(quartile_1_no)

sub_category
Air Conditioners                3.00
All Exercise & Fitness          8.75
All Grocery & Gourmet Foods    82.50
Camping & Hiking                2.00
Cardio Equipment                3.00
Fashion & Silver Jewellery      3.00
Fashion Sandals                 2.00
Household Supplies              4.00
Indoor Lighting                 6.00
Kitchen & Dining               26.00
Lab & Scientific                2.00
Men's Fashion                   9.00
Sewing & Craft Supplies         6.00
Snack Foods                     7.00
Sports Shoes                    4.00
Strength Training               4.00
Toys & Games                   15.00
Travel Accessories              3.00
Travel Duffles                  3.00
Watches                         2.00
Western Wear                    4.00
Name: no_of_ratings, dtype: float64


In [56]:
# Group by 'sub_category' and calculate the 1/4th quartile for 'no_of_ratings'
quartile_1_rating = grouped['ratings'].quantile(0.75)
# Print the 1/4th quartile values
print(quartile_1_rating)

sub_category
Air Conditioners               4.2
All Exercise & Fitness         4.2
All Grocery & Gourmet Foods    4.3
Camping & Hiking               4.5
Cardio Equipment               4.3
Fashion & Silver Jewellery     4.3
Fashion Sandals                4.1
Household Supplies             4.5
Indoor Lighting                4.3
Kitchen & Dining               4.3
Lab & Scientific               4.6
Men's Fashion                  4.2
Sewing & Craft Supplies        4.4
Snack Foods                    4.3
Sports Shoes                   4.1
Strength Training              4.2
Toys & Games                   4.3
Travel Accessories             4.4
Travel Duffles                 4.5
Watches                        4.4
Western Wear                   4.1
Name: ratings, dtype: float64


In [59]:
# # Create scatter plots for each sub_category
# for sub_category, group in grouped:
#     plt.figure(figsize=(8, 6))
#     plt.scatter(group['no_of_ratings'], group['ratings'], alpha=0.6)
#     plt.title(f'Scatter Plot for {sub_category}')
#     plt.xlabel('Number of Ratings')
#     plt.ylabel('Ratings')
#     plt.grid(True)
#     plt.show()

In [58]:
def apply_thresholds(row):
    category = row['sub_category']
    return (row['no_of_ratings'] < quartile_1_no[category]) and (row['ratings'] > quartile_1_rating[category])

df_filtered = df[df.apply(apply_thresholds, axis=1)]
print(df_filtered.shape)

(7690, 10)


In [60]:
df_filtered.to_csv("Hidden products.csv", index=False)